# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
pokemon = pd.read_csv('data/pokemon_updated.csv')

From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [3]:
#Total = Sum(HP + Attack + Defense + Sp. Atk + Sp. Def + Speed)

result = pokemon["Total"] == pokemon[["HP","Attack","Defense","Sp. Atk","Sp. Def","Speed"]].sum(axis=1)
print(type(result))

#check if panda series contains other values than True
print(set(result))

#No false, so for formula works for all pokemon

<class 'pandas.core.series.Series'>
{True}


## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

[One Hot Encoding](https://drive.google.com/file/d/1McE3eNAyUxBXHfEvyTTz-ZrMDB7-U3_z/view?usp=sharing)

In [4]:
one_hot_coding = pd.get_dummies(pokemon["Type 1"]) + pd.get_dummies(pokemon["Type 2"])

#check first 10
print(one_hot_coding.head(10))
print('\n')
print(pokemon.head(10))

   Bug  Dark  Dragon  Electric  Fairy  Fighting  Fire  Flying  Ghost  Grass  \
0    0     0       0         0      0         0     0       0      0      1   
1    0     0       0         0      0         0     0       0      0      1   
2    0     0       0         0      0         0     0       0      0      1   
3    0     0       0         0      0         0     0       0      0      1   
4    0     0       0         0      0         0     1       0      0      0   
5    0     0       0         0      0         0     1       0      0      0   
6    0     0       0         0      0         0     1       1      0      0   
7    0     0       1         0      0         0     1       0      0      0   
8    0     0       0         0      0         0     1       1      0      0   
9    0     0       0         0      0         0     0       0      0      0   

   Ground  Ice  Normal  Poison  Psychic  Rock  Steel  Water  
0       0    0       0       1        0     0      0      0  
1     

## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [5]:
print(one_hot_coding.corrwith(pokemon['Total']).sort_values(ascending=False))
#Dragon scores highest, but not high correlation score

#Check scores of Dragon and Psychic
filtered = pokemon[(pokemon['Type 1'] == 'Dragon')]
print('\n')
print('Average Total Score of Dragon Pokemons:')
print(filtered['Total'].mean())
filtered2 = pokemon[(pokemon['Type 1'] == 'Psychic')]
print('\n')
print('Average Total Score of Psychic Pokemons:')
print(filtered2['Total'].mean())
#Dragon scores a lot better than Psychic

Dragon      0.229705
Psychic     0.124688
Steel       0.109703
Fire        0.078726
Fighting    0.077786
Ice         0.060248
Flying      0.059383
Dark        0.056154
Rock        0.032731
Electric    0.020971
Ground      0.015060
Ghost       0.003641
Water      -0.021665
Fairy      -0.036698
Grass      -0.052592
Poison     -0.090441
Normal     -0.105331
Bug        -0.145781
dtype: float64


Average Total Score of Dragon Pokemons:
550.53125


Average Total Score of Psychic Pokemons:
475.94736842105266


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [6]:
#add combo type column of challenge 1 to df
pokemon['Combo Type'] = np.where(pokemon['Type 2'].isnull(), pokemon['Type 1'], pokemon['Type 1'] + '-' + pokemon['Type 2'])

#perform one hot coding for combo type
one_hot_coding2 = pd.get_dummies(pokemon["Combo Type"])

#calculate correlation with combo type
print('Correlation Combo Type with Total:')
print(one_hot_coding2.corrwith(pokemon['Total']).sort_values(ascending=False))
print('\n')
#Dragon-Flying pokemon

#Check with highest Total
print('Pokemons with highest Total score:')
print(pokemon.nlargest(5, ['Total']))
print('\n')
print('Dragon-Flying average score:')
filtered = pokemon[(pokemon['Type 1'] == 'Dragon') & (pokemon['Type 2'] == 'Flying')]
print(filtered['Total'].mean())
print('\n')
filtered = pokemon[(pokemon['Type 1'] == 'Dragon') & (pokemon['Type 2'] == 'Ice')]
print('Dragon-Ice average score:')
print(filtered['Total'].mean())

#On correlation score you would choice Dragon-Flying, both Dragon-Ice scores better!

Correlation Combo Type with Total:
Dragon-Flying       0.149777
Dragon-Ice          0.128737
Dragon-Psychic      0.127066
Psychic-Fighting    0.104173
Ghost-Dragon        0.102264
                      ...   
Poison             -0.064380
Normal-Fairy       -0.069525
Bug-Poison         -0.089742
Normal-Flying      -0.092626
Bug                -0.178699
Length: 154, dtype: float64


Pokemons with highest Total score:
                      Name   Type 1    Type 2  Total   HP  Attack  Defense  \
163    MewtwoMega Mewtwo X  Psychic  Fighting    780  106     190      100   
164    MewtwoMega Mewtwo Y  Psychic       NaN    780  106     150       70   
426  RayquazaMega Rayquaza   Dragon    Flying    780  105     180      100   
422    KyogrePrimal Kyogre    Water       NaN    770  100     150       90   
424  GroudonPrimal Groudon   Ground      Fire    770  100     180      160   

     Sp. Atk  Sp. Def  Speed  Generation  Legendary        Combo Type  
163      154      100    130           1